# Домашнее задание № 1

Вам необходимо выбор стратегии валидации и обработку данных от ошибок заполнения.

# Часть 3. Стратегия валидации

Итак, у вас есть данные для обучения и данные, на которых необходимо сделать предсказание. Для оценки точности необходимо проводить валидацию (так как мало данных, можно проводить кросс валидацию).
Валидацию проще всего реализовать методами sklearn, например:
- KFold
- ShuffleSplit
- StratifiedKFold
- StratifiedShuffleSplit
- GroupKFold
- GroupShuffleSplit

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.model_selection import KFold, ShuffleSplit, StratifiedKFold, StratifiedShuffleSplit, GroupKFold, GroupShuffleSplit

In [26]:
X_train = pd.read_csv('train_features_with_answers.csv')
X_train.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,MS,M,NaN,U,LE3,T,1,3,at_home,services,...,yes,yes,4,3,3,2,3,3,0,9
1,GP,M,NaN,R,LE3,T,3,3,teacher,other,...,yes,no,3,3,4,3,5,3,16,12
2,GP,M,15.0,NaN,GT3,T,3,4,other,other,...,yes,no,5,5,1,1,1,5,0,13
3,MS,M,17.0,R,GT3,T,4,3,services,other,...,yes,yes,4,5,5,1,3,2,4,11
4,GP,F,20.0,R,GT3,T,2,1,other,other,...,yes,yes,1,2,3,1,2,2,8,12


#### Примеры разбиений (раскомментить нужную строку для стратегии)

In [31]:
# CV strategy

# cv_folds = KFold(n_splits=5)
# cv_folds = ShuffleSplit(n_splits=5)
# cv_folds = StratifiedKFold(n_splits=5)
# cv_folds = StratifiedShuffleSplit(n_splits=5)
# cv_folds = GroupKFold(n_splits=5)
cv_folds = GroupShuffleSplit(n_splits=5)
'Group' not in type(cv_folds).__name__

False

In [32]:
# CV example
fold_collection = {}
# base
if 'Stratified' not in type(cv_folds).__name__ and 'Group' not in type(cv_folds).__name__:
    for i, (train_index, test_index) in enumerate(cv_folds.split(X_train)):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")
        fold_collection[i] = {
            "Train": [X_train.loc[:, X_train.columns != 'G3'].iloc[train_index], X_train.loc[:, X_train.columns == 'G3'].iloc[train_index]],
            "Test": [X_train.loc[:, X_train.columns != 'G3'].iloc[test_index], X_train.loc[:, X_train.columns == 'G3'].iloc[test_index]],
        }


# stratified
if 'Stratified' in type(cv_folds).__name__:
    design, target = X_train.loc[:, X_train.columns != 'G3'], X_train['G3']
    for i, (train_index, test_index) in enumerate(cv_folds.split(design, target)):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")
        fold_collection[i] = {
            "Train": [X_train.loc[:, X_train.columns != 'G3'].iloc[train_index], X_train.loc[:, X_train.columns == 'G3'].iloc[train_index]],
            "Test": [X_train.loc[:, X_train.columns != 'G3'].iloc[test_index], X_train.loc[:, X_train.columns == 'G3'].iloc[test_index]],
        }

        print('train -  {}   |   test -  {}'.format(np.bincount(target[train_index]), np.bincount(target[test_index])))
        
# group
if 'Group' in type(cv_folds).__name__:
    design, target = X_train.loc[:, X_train.columns != 'G3'], X_train['G3']
    group = design["higher"]
    for i, (train_index, test_index) in enumerate(cv_folds.split(design, target, groups = group)):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")
        fold_collection[i] = {
            "Train": [X_train.loc[:, X_train.columns != 'G3'].iloc[train_index], X_train.loc[:, X_train.columns == 'G3'].iloc[train_index]],
            "Test": [X_train.loc[:, X_train.columns != 'G3'].iloc[test_index], X_train.loc[:, X_train.columns == 'G3'].iloc[test_index]],
        }

fold_collection

Fold 0:
  Train: index=[  0   4  16  21  22  34  47  56  58  61  72  79  84 103 118 123 126 128
 131 136 139 152 156 176 179 191 209 213 220 227 230 245 246 258 260 261
 266 279 288 292 314 334 356 385 393 409 412 414 418 434]
  Test:  index=[  1   2   3   5   6   7   8   9  10  11  12  13  14  15  17  18  19  20
  23  24  25  26  27  28  29  30  31  32  33  35  36  37  38  39  40  41
  42  43  44  45  46  48  49  50  51  52  53  54  55  57  59  60  62  63
  64  65  66  67  68  69  70  71  73  74  75  76  77  78  80  81  82  83
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 119 120 121 122
 124 125 127 129 130 132 133 134 135 137 138 140 141 142 143 144 145 146
 147 148 149 150 151 153 154 155 157 158 159 160 161 162 163 164 165 166
 167 168 169 170 171 172 173 174 175 177 178 180 181 182 183 184 185 186
 187 188 189 190 192 193 194 195 196 197 198 199 200 201 202 203 204 205
 206 207 208 210 211 212 214

{0: {'Train': [    school sex   age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
   0       MS   M   NaN       U     LE3       T     1     3   at_home  services   
   4       GP   F  20.0       R     GT3       T     2     1     other     other   
   16      MS   F  18.0       R     GT3       T     1     1   at_home   at_home   
   21      GP   M  17.0     NaN     LE3       T     1     1     other  services   
   22      GP   F  16.0     NaN     GT3       T     1     2     other     other   
   34      GP   M  18.0       U     LE3       T     2     2  services  services   
   47      GP   F  19.0       R     GT3       T     3     2  services  services   
   56      GP   M  18.0       U     GT3       T     2     1  services  services   
   58      GP   M  17.0       U     GT3       T     2     2     other     other   
   61      GP   F  19.0       U     GT3       T     1     1     other     other   
   72      MS   F  19.0       U     LE3       A     1     1   at_home     o

## Разделите данные и объясните выбор стратегии

In [ ]:
# Ваш код


__Ваш ответ:__ лучше бить по дефолту через _StratifiedShuffleSplit_ для классификации, тк мы не знаем, обьединены ли данные одной природой, чтобы использовать GroupFold, и лучше перемешать данные для генерации независимой выборки, а так сохранится доля классов в test train и будет сохраняться распределение классов. 

Данную задачу можно рассматривать и как регрессию, и как классификацию. Для регрессии сложно взять Stratified тк нету деления по классам, поэтому лучше выбрать _ShuffleSplit_

Если в данных есть какая-то группировка по природе объектов - лучше использовать Group strategy. Возможно объекты сгруппированы по Fedu, higher и studytime (высокая корреляция с G3). Поэтому можно сделать группы с высшим образованием (_higher_ - бинарный признак, лекго делить)

__Вывод:__ Стоит попробовать Group Shuffle на higher и либо StratifiedShuffleSplit (если рассматривать как классификацию), либо 

# Часть 4. Заполнение пропусков в данных 

## Фильтрация по диапазону значений 

Проверьте, что все характеристики принадлежат тому диапазону значений, который приведен в описании к исходным данным. Например, если в графе "Пол" есть значение FF, P или -1, то это явная ошибка заполнения.

Приведите данные в соответствии с описанием данных. Если значение указано неверно - укажите ```NaN```.

In [ ]:
# Ваш код

## Адаптивное заполнения пропусков 

Реализуйте стратегию заполнения пропусков данных на основе схожих примеров. Обучите один из известных вам алгоритмов (knn, Naive Bayes, linear regression) для предсказания той характеристики, пропуски которой присутсвуют в данных. В качестве характеристики для обучения алгоритмов __нельзя__ использовать ```G3```.

In [ ]:
# Ваш код